In [ ]:
from numcosmo_py import Ncm, Nc, GObject

import numpy as np
import math
import sys
from astropy.io import fits
from astropy.table import Table
import matplotlib.pyplot as plt

from richness_mass_calib import create_richness_mass_calib

In [ ]:
# #Settings
Ncm.cfg_init()
Ncm.cfg_set_log_handler(lambda msg: sys.stdout.write(msg) and sys.stdout.flush())

In [ ]:
# Data
DC2_halos_m200c = fits.open(
    "/global/cfs/projectdirs/lsst/groups/CL/cosmoDC2_v1.1.4/extragal/full/halos/halos_m200c_13.0.fits"
)
dt_halos = Table(DC2_halos_m200c[1].data)
# dt_halos['redshift_true', 'mass_fof', 'm200c', 'richness']

In [ ]:
rmdata = create_richness_mass_calib(dt_halos)
ascaso = Nc.ClusterMassAscaso()
lnrich_ext = Nc.ClusterMassLnrichExt()

mset = Ncm.MSet()
mset.set(ascaso)

rmdata.m2lnL_val(mset)

In [ ]:
dset = Ncm.Dataset.new()
dset.append_data(rmdata)

lh = Ncm.Likelihood.new(dset)
mset.param_set_all_ftype(Ncm.ParamType.FREE)

mset.prepare_fparam_map()


fit = Ncm.Fit.new(
    Ncm.FitType.NLOPT, "ln-neldermead", lh, mset, Ncm.FitGradType.NUMDIFF_CENTRAL
)

fit.run_restart(Ncm.FitRunMsgs.SIMPLE, 1.0e-3, 0.0, None, None)

In [ ]:
fit.log_info()

In [ ]:
nwalkers = 500

Ncm.func_eval_set_max_threads(2)
Ncm.func_eval_log_pool_stats()

init_sampler = Ncm.MSetTransKernGauss.new(0)
init_sampler.set_mset(mset)
init_sampler.set_prior_from_mset()
init_sampler.set_cov_from_rescale(1.0)

apes = Ncm.FitESMCMCWalkerAPES.new(nwalkers, mset.fparams_len())

esmcmc = Ncm.FitESMCMC.new(fit, nwalkers, init_sampler, apes, Ncm.FitRunMsgs.FULL)
esmcmc.set_nthreads(2)
esmcmc.set_data_file("ascaso1.fits")

esmcmc.start_run()
esmcmc.run(1000)  
esmcmc.end_run()

esmcmc.mean_covar()
fit.log_covar()

In [ ]:
esmcmc.mean_covar()
fit.log_covar()

In [ ]:
fit.log_covar()

In [ ]:
fit.log_info()